# Part 4. Matrix and Vector Computations

13. Read about Broadcasting with Arrays on the chapter Computation on Arrays: 
Broadcasting from Python Data Science Handbook (J. VandePlas, 2016): 
Link: https://jakevdp.github.io/PythonDataScienceHandbook/02.05-computation
on-arrays-broadcasting.html. 

NumPy offers a range of powerful features designed to enhance computational efficiency when working with arrays. One of the key components facilitating this efficiency is the concept of universal functions, commonly referred to as ufuncs. These ufuncs allow for fast array computations by supporting both unary and binary operations. This includes a variety of mathematical functions such as addition, subtraction, trigonometric calculations, and more. By using ufuncs, NumPy can perform these operations in a vectorized manner, which means it can handle computations on entire arrays at once, bypassing the slower process of looping through each element in Python. This vectorized approach significantly boosts performance, making it a valuable asset for numerical computing tasks.

In addition to basic mathematical operations, ufuncs come with specialized features that further enhance their utility. These features include the ability to specify output, perform aggregates, and compute outer products. Such functionalities not only streamline the coding process but also contribute to improved computational efficiency, making NumPy a preferred tool for high-performance computing (HPC) applications.

Alongside ufuncs, broadcasting is another standout feature in NumPy that contributes to efficient array operations. Broadcasting allows for vectorized operations to be performed across arrays with different shapes without the need for explicit data replication or reshaping. Instead of requiring arrays to have identical shapes, broadcasting intelligently aligns arrays by extending or stretching them as needed. This simplifies the code and enhances computational and memory efficiency, which is particularly important for HPC tasks. NumPy follows specific rules to handle operations between arrays with mismatched shapes, such as extending dimensions of size one to match corresponding dimensions of the other array. These rules uphold the principles of efficiency by eliminating unnecessary data duplication and promoting optimal computational practices.


14. Read Rewriting the particle simulator in Numpy on Chapter 2: Fast Array Operations 
with Numpy and Pandas (pp. 68) from the book G. Lenaro (2017). Python high 
Performance. Second Edition. UK: Packt Publishing Ltd. Implement the 
improvements on the particle simulator using NumPy. Show that both 
implementations scale linearly with particle size, but the runtime in the pure Python 
version grows much faster than the NumPy version.  

In [1]:
import numpy as np
from random import uniform
from timeit import timeit
import numexpr as ne

In [2]:
class Particle:
    def __init__(self, x, y, velocidad_angular):
        self.x = x
        self.y = y
        self.velocidad_angular = velocidad_angular

class ParticleSimulator:

    def __init__(self, particles):
        self.particles = particles

    def evolve_python(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        for i in range(nsteps):
            for p in self.particles:
                norm = (p.x**2 + p.y**2)**0.5
                v_x = (-p.y) / norm
                v_y = p.x / norm
                d_x = timestep * p.velocidad_angular * v_x
                d_y = timestep * p.velocidad_angular * v_y
                p.x += d_x
                p.y += d_y

    def evolve_numpy(self, dt):
        timestep = 0.00001
        nsteps = int(dt / timestep)
        r_i = np.array([[p.x, p.y] for p in self.particles])
        velocidad_angular_i = np.array([p.velocidad_angular for p in self.particles])
        for i in range(nsteps):
            norm_i = np.sqrt((r_i ** 2).sum(axis=1))
            v_i = r_i[:, [1, 0]] * np.array([-1, 1]) / norm_i[:, np.newaxis]
            d_i = timestep * velocidad_angular_i[:, np.newaxis] * v_i
            r_i += d_i
        for i, p in enumerate(self.particles):
            p.x, p.y = r_i[i]

 
def benchmark(npart=1000, method='python'):
    particles = [Particle(uniform(-1.0, 1.0), uniform(-1.0, 1.0), uniform(-1.0, 1.0)) for _ in range(npart)]
    simulator = ParticleSimulator(particles)
    if method == 'python':
        simulator.evolve_python(0.1)
    elif method == 'numpy':
        simulator.evolve_numpy(0.1)

 
# Example of benchmarking
if __name__ == "__main__":

    print("Benchmarking with 100000 particles:")
    print("Pure python:")
    print(timeit("benchmark(1000, 'python')", globals=globals(), number=1))
    print("Numpy:")
    print(timeit("benchmark(1000, 'numpy')", globals=globals(), number=1))
                

Benchmarking with 100000 particles:
Pure python:
5.061089099999663
Numpy:
0.41473410000071453


15. Explain how to optain the optimal performance with numexpr. Read the section 
Reaching optimal performance with numexpr, pp. 72 from the previous reference. 
Implement it and measure the execution time.

In [3]:
a = np.arange(1e6)
b = np.arange(1e6)

In [4]:
%%timeit -n200 -r10
c = a**2 + b**2 

6.41 ms ± 246 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)


In [5]:
%%timeit -n200 -r10
c = ne.evaluate("a**2 + b**2")

3.85 ms ± 196 µs per loop (mean ± std. dev. of 10 runs, 200 loops each)
